In [1]:
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
import torch
import numpy as np

In [4]:
class GCN_N_layer(nn.Module):
    def __init__(self, A, N=0, F = 1433, class_number=7, hidden_neurons=200):
        super(GCN_N_layer, self).__init__()
        self.A = A
        self.class_number = class_number
        self.N = N
        self.hidden = nn.Linear(hidden_neurons, hidden_neurons, bias=True)
        self.fc1 = nn.Linear(F, hidden_neurons, bias=True)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(200, self.class_number, bias=True)

    def forward(self, x):
        # training on full x, not batch
        x = x.float()
        # average all neighboors
        #print(x.shape)
        #A = self.A.float()
        #print(A.shape)
        #print(self.X.shape)
        #print(A.dtype, self.X.dtype, x.dtype)
        x = torch.matmul(self.A, x)
        x = self.fc1(x)
        x = self.relu(x)
        for i in range(self.N):
            x = self.hidden(x)
            x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        return x

class n_hidden_GCN():
    def __init__(self, A, X, y, device="cuda", N=0, F=1433, class_number = 7, hidden_neurons=200, self_weight=10, val_size=0.3):
        self.device = torch.device(device)
        le = preprocessing.LabelEncoder()
        le.fit(y)
        y = le.transform(y)
        
        X = torch.tensor(X)
        X = X.type(torch.float)
        y = torch.tensor(y)
        y = y.type(torch.long)
        y = y.to(self.device)
        
        if A[0][0] == 0:
            A = A * 1./self_weight + np.identity(A.shape[0])
        A = torch.from_numpy(A).float()
        
        self.X = X.to(self.device)
        self.A = A.to(self.device)
        self.y = y.to(self.device)
    
        self.model = GCN_N_layer(A=self.A, N=N, F=F, class_number=class_number, hidden_neurons=hidden_neurons)
        
        train_idx = np.random.choice(self.X.shape[0], round(self.X.shape[0]*(1-val_size)), replace=False)
        val_idx = np.array([x for x in range(X.shape[0]) if x not in train_idx])
        print("Train length :{a}, Validation length :{b}".format(a=len(train_idx), b=len(val_idx)))
        
        self.idx_train = torch.LongTensor(train_idx)
        self.idx_val = torch.LongTensor(val_idx)
        self.model.to(self.device)
        
    def train(self, optimizer, epoch):
        self.model.train()
        optimizer.zero_grad()
        output = self.model(self.X)
        loss = F.cross_entropy(output[self.idx_train], self.y[self.idx_train])
        loss.backward(retain_graph=True)
        optimizer.step()
        print('Epoch: {x}'.format(x=epoch))
        print('training loss {:.4f}'.format(loss.item()))
            
    def test(self):
        self.model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            output = self.model(self.X)
            loss = F.cross_entropy(output[self.idx_val], self.y[self.idx_val], reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)[self.idx_val]
            correct += pred.eq(self.y[self.idx_val].view_as(pred)).sum().item()

        test_loss /= len(self.idx_val)
        print('Validtion: Average loss: {:.4f}, Accuracy: {:.4f}%'.format(test_loss, 100. * correct / len(self.idx_val)))
        return 100. * correct / len(self.idx_val)
    
    def train_epoch(self, epochs=100, lr=1e-3):
        acc = []
        optimizer = optim.Adam(self.model.parameters(), lr=lr)#, weight_decay=1e-1)

        for epoch in range(epochs):
            self.train(optimizer, epoch)
            accs = self.test()
            acc.append(accs)
        accs = {'acc': acc}
        return accs

In [5]:
from coraloader import cora_loader

In [6]:
cora = cora_loader('data' + '/cora.content', 'data' + '/cora.cites', None)
X, y, A = cora.get_train()

In [7]:
model = n_hidden_GCN(A, X, y)

Train length :1896, Validation length :812


In [8]:
model.train_epoch()

Epoch: 0
training loss 1.9447
Validtion: Average loss: 0.0000, Accuracy: 37.5616%
Epoch: 1
training loss 1.9193
Validtion: Average loss: 0.0000, Accuracy: 39.7783%
Epoch: 2
training loss 1.8938
Validtion: Average loss: 0.0000, Accuracy: 38.6700%
Epoch: 3
training loss 1.8668
Validtion: Average loss: 0.0000, Accuracy: 38.7931%
Epoch: 4
training loss 1.8377
Validtion: Average loss: 0.0000, Accuracy: 38.5468%
Epoch: 5
training loss 1.8063
Validtion: Average loss: 0.0000, Accuracy: 40.1478%
Epoch: 6
training loss 1.7726
Validtion: Average loss: 0.0000, Accuracy: 41.3793%
Epoch: 7
training loss 1.7368
Validtion: Average loss: 0.0000, Accuracy: 42.7340%
Epoch: 8
training loss 1.6990
Validtion: Average loss: 0.0000, Accuracy: 44.8276%
Epoch: 9
training loss 1.6595
Validtion: Average loss: 0.0000, Accuracy: 47.5369%
Epoch: 10
training loss 1.6184
Validtion: Average loss: 0.0000, Accuracy: 50.2463%
Epoch: 11
training loss 1.5760
Validtion: Average loss: 0.0000, Accuracy: 53.2020%
Epoch: 12
trai

{'acc': [37.5615763546798,
  39.77832512315271,
  38.669950738916256,
  38.793103448275865,
  38.54679802955665,
  40.14778325123153,
  41.37931034482759,
  42.73399014778325,
  44.827586206896555,
  47.53694581280788,
  50.24630541871921,
  53.20197044334975,
  56.89655172413793,
  58.86699507389162,
  60.960591133004925,
  63.05418719211823,
  64.28571428571429,
  67.36453201970443,
  69.45812807881774,
  70.93596059113301,
  72.41379310344827,
  73.39901477832512,
  74.26108374384236,
  74.8768472906404,
  76.10837438423646,
  76.9704433497537,
  77.83251231527093,
  78.32512315270937,
  78.94088669950739,
  79.43349753694581,
  79.92610837438424,
  80.91133004926108,
  81.52709359605912,
  81.77339901477832,
  81.52709359605912,
  82.26600985221675,
  83.00492610837438,
  82.75862068965517,
  83.3743842364532,
  83.12807881773399,
  82.88177339901478,
  83.00492610837438,
  82.88177339901478,
  83.2512315270936,
  83.00492610837438,
  83.00492610837438,
  83.00492610837438,
  83.12